# Comprehensive Guide to Building AI Agents with LangChain

Welcome to the complete hands-on course for building AI agents using LangChain and LangGraph! This notebook will guide you through creating intelligent agents that can reason, use tools, and solve complex real-world problems.

## 🎯 Course Overview

By the end of this course, you'll understand how to:
- Build custom tools and integrate them with LLMs
- Create different agent architectures (ReAct, Structured Chat, LangGraph)
- Implement real-world applications for finance, research, and customer support
- Debug, monitor, and optimize agent performance
- Handle complex multi-step workflows with memory and state management

## 📚 What You'll Learn

### Module Structure:
1. **Environment Setup** - Configure your development environment
2. **Tool Creation** - Build custom tools with proper schemas
3. **Agent Architectures** - Implement ReAct, Structured Chat, and LangGraph agents  
4. **Real-World Applications** - Financial agents, research assistants, and more
5. **Best Practices** - Debugging, monitoring, and optimization techniques

Let's start building intelligent AI agents! 🚀

## 1. Environment Setup and Library Installation

First, let's install and import all the required libraries for building AI agents.

In [ ]:
# Install required packages (run this if packages are not installed)
# !pip install langchain==0.1.0 langchain-community==0.0.13 langchain-openai==0.0.2 
# !pip install langgraph==0.0.20 openai==1.10.0 requests numpy pandas wikipedia python-dotenv

# Import core libraries
import os
import json
import requests
import numpy as np
import pandas as pd
from typing import Dict, List, Any, Optional
from datetime import datetime

# Import LangChain components
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

# Import LangGraph for advanced orchestration
try:
    from langgraph.prebuilt import create_react_agent
    print("✅ LangGraph imported successfully")
except ImportError:
    print("❌ LangGraph not available - will use alternative implementations")

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("🎉 All libraries imported successfully!")
print(f"📅 Session started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Configure API Keys (set your actual API keys in .env file)
# For this demo, we'll use mock implementations when API keys are not available

def setup_llm():
    """Setup LLM with proper API key handling"""
    openai_api_key = os.getenv("OPENAI_API_KEY")
    
    if openai_api_key and openai_api_key != "your_openai_api_key_here":
        # Use real OpenAI
        llm = ChatOpenAI(
            model="gpt-3.5-turbo",
            temperature=0,
            api_key=openai_api_key
        )
        print("✅ Using real OpenAI LLM")
        return llm, True
    else:
        # Mock LLM for demonstration purposes
        print("⚠️  Using mock LLM (set OPENAI_API_KEY for real functionality)")
        return None, False

# Initialize LLM
llm, has_real_llm = setup_llm()

print(f"🤖 LLM Status: {'Real LLM configured' if has_real_llm else 'Mock LLM (demo mode)'}")
print("💡 Tip: Add your OpenAI API key to .env file for full functionality")

## 2. Basic Tool Creation and Function Calling

Tools are the bridge between LLMs and the real world. Let's create our first custom tools with proper schemas and type hints.

In [ ]:
@tool
def simple_calculator(expression: str) -> str:
    """
    Evaluate a simple mathematical expression safely.
    
    Args:
        expression: Mathematical expression as string (e.g., "2 + 3 * 4")
        
    Returns:
        Result of the calculation as string
        
    Example:
        simple_calculator("10 + 5") → "15"
    """
    try:
        # Safe evaluation - only allow specific characters
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def weather_simulator(location: str) -> Dict[str, Any]:
    """
    Simulate weather API call (mock implementation for demo).
    
    Args:
        location: City name or location (e.g., "New York", "London")
        
    Returns:
        Dictionary with weather information
    """
    import random
    
    # Mock weather data
    conditions = ["sunny", "cloudy", "rainy", "snowy", "partly cloudy"]
    
    return {
        "location": location,
        "temperature": random.randint(50, 85),
        "condition": random.choice(conditions),
        "humidity": random.randint(30, 90),
        "source": "mock_weather_api",
        "timestamp": datetime.now().isoformat()
    }

# Test our tools
print("🧮 Testing Calculator Tool:")
calc_result = simple_calculator.invoke({"expression": "15 + 27 * 2"})
print(f"   15 + 27 * 2 = {calc_result}")

print("\n🌤️  Testing Weather Tool:")
weather_result = weather_simulator.invoke({"location": "San Francisco"})
print(f"   Weather in {weather_result['location']}: {weather_result['temperature']}°F, {weather_result['condition']}")

print("\n✅ Basic tools created and tested successfully!")

## 3. Building Math Toolkit with Custom Tools

Let's create a comprehensive math toolkit with proper type hints and error handling.

In [ ]:
@tool
def add_numbers(numbers: List[float]) -> Dict[str, Any]:
    """
    Add a list of numbers with detailed result information.
    
    Args:
        numbers: List of numbers to add (e.g., [1, 2, 3, 4])
        
    Returns:
        Dictionary with sum and metadata
    """
    if not numbers:
        return {"error": "Empty list provided", "sum": 0}
    
    try:
        total = sum(numbers)
        return {
            "operation": "addition",
            "input_numbers": numbers,
            "sum": total,
            "count": len(numbers),
            "average": total / len(numbers),
            "min_value": min(numbers),
            "max_value": max(numbers)
        }
    except Exception as e:
        return {"error": f"Addition failed: {str(e)}"}

@tool 
def multiply_numbers(numbers: List[float]) -> Dict[str, Any]:
    """
    Multiply a list of numbers with comprehensive analysis.
    
    Args:
        numbers: List of numbers to multiply (e.g., [2, 3, 4])
        
    Returns:
        Dictionary with product and analysis
    """
    if not numbers:
        return {"error": "Empty list provided", "product": 1}
    
    try:
        product = float(np.prod(numbers))
        
        return {
            "operation": "multiplication",
            "input_numbers": numbers,
            "product": product,
            "count": len(numbers),
            "contains_zero": 0 in numbers,
            "all_positive": all(n > 0 for n in numbers)
        }
    except Exception as e:
        return {"error": f"Multiplication failed: {str(e)}"}

@tool
def statistical_analysis(numbers: List[float]) -> Dict[str, Any]:
    """
    Perform comprehensive statistical analysis on numbers.
    
    Args:
        numbers: List of numbers to analyze
        
    Returns:
        Dictionary with statistical measures
    """
    if not numbers:
        return {"error": "No data provided"}
    
    try:
        np_array = np.array(numbers)
        
        return {
            "operation": "statistical_analysis",
            "input_numbers": numbers,
            "count": len(numbers),
            "mean": float(np.mean(np_array)),
            "median": float(np.median(np_array)),
            "std_deviation": float(np.std(np_array)),
            "variance": float(np.var(np_array)),
            "min": float(np.min(np_array)),
            "max": float(np.max(np_array)),
            "range": float(np.max(np_array) - np.min(np_array))
        }
    except Exception as e:
        return {"error": f"Statistical analysis failed: {str(e)}"}

# Test the math toolkit
print("🧮 Testing Math Toolkit:")

test_numbers = [10, 20, 30, 40, 50]
print(f"\nTest data: {test_numbers}")

# Test addition
add_result = add_numbers.invoke({"numbers": test_numbers})
print(f"✅ Addition: Sum = {add_result['sum']}, Average = {add_result['average']}")

# Test multiplication  
mult_result = multiply_numbers.invoke({"numbers": [2, 3, 4]})
print(f"✅ Multiplication: Product = {mult_result['product']}")

# Test statistics
stats_result = statistical_analysis.invoke({"numbers": test_numbers})
print(f"✅ Statistics: Mean = {stats_result['mean']:.1f}, Std Dev = {stats_result['std_deviation']:.1f}")

print("\n🎯 Math toolkit ready for agent integration!")

## 4. Creating Wikipedia Research Tools

Now let's add research capabilities by creating Wikipedia query tools and combining them with our math tools.

In [ ]:
@tool
def wikipedia_search(query: str, max_results: int = 3) -> Dict[str, Any]:
    """
    Search Wikipedia for information (mock implementation for demo).
    
    Args:
        query: Search query string
        max_results: Maximum number of results to return
        
    Returns:
        Dictionary with search results and metadata
    """
    # Mock Wikipedia data for demonstration
    mock_articles = {
        "python": {
            "title": "Python (programming language)",
            "summary": "Python is a high-level, interpreted programming language with dynamic semantics.",
            "url": "https://en.wikipedia.org/wiki/Python_(programming_language)"
        },
        "machine learning": {
            "title": "Machine Learning", 
            "summary": "Machine learning is a method of data analysis that automates analytical model building.",
            "url": "https://en.wikipedia.org/wiki/Machine_learning"
        },
        "artificial intelligence": {
            "title": "Artificial Intelligence",
            "summary": "Artificial intelligence is intelligence demonstrated by machines.",
            "url": "https://en.wikipedia.org/wiki/Artificial_intelligence"
        },
        "langchain": {
            "title": "LangChain Framework",
            "summary": "LangChain is a framework for developing applications powered by language models.",
            "url": "https://en.wikipedia.org/wiki/LangChain"
        }
    }
    
    # Find relevant articles
    query_lower = query.lower()
    results = []
    
    for key, article in mock_articles.items():
        if key in query_lower or any(word in key for word in query_lower.split()):
            results.append(article)
            if len(results) >= max_results:
                break
    
    # Generate generic result if no specific match
    if not results:
        results = [{
            "title": f"Search results for '{query}'",
            "summary": f"Mock search result about {query}. This would contain relevant information from Wikipedia.",
            "url": f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"
        }]
    
    return {
        "query": query,
        "results_found": len(results),
        "results": results[:max_results],
        "source": "mock_wikipedia_api",
        "timestamp": datetime.now().isoformat()
    }

@tool
def knowledge_synthesizer(research_data: str, numbers: List[float] = None) -> Dict[str, Any]:
    """
    Combine research information with numerical analysis.
    
    Args:
        research_data: Text data from research
        numbers: Optional numerical data for analysis
        
    Returns:
        Dictionary with synthesized insights
    """
    synthesis = {
        "research_summary": research_data[:200] + "..." if len(research_data) > 200 else research_data,
        "word_count": len(research_data.split()),
        "key_topics": [word for word in research_data.lower().split() 
                      if len(word) > 6 and word.isalpha()][:5]
    }
    
    if numbers:
        synthesis["numerical_analysis"] = {
            "data_points": len(numbers),
            "average": sum(numbers) / len(numbers) if numbers else 0,
            "total": sum(numbers) if numbers else 0
        }
    
    return synthesis

# Test the research tools
print("🔍 Testing Wikipedia Search Tool:")

search_result = wikipedia_search.invoke({"query": "machine learning", "max_results": 2})
print(f"Search query: '{search_result['query']}'")
print(f"Results found: {search_result['results_found']}")

for i, result in enumerate(search_result['results'], 1):
    print(f"  {i}. {result['title']}")
    print(f"     {result['summary'][:100]}...")

print("\n🧠 Testing Knowledge Synthesizer:")
sample_text = "Machine learning is a powerful tool for data analysis and prediction."
synthesis = knowledge_synthesizer.invoke({
    "research_data": sample_text,
    "numbers": [85, 92, 78, 90, 88]
})

print(f"Key topics: {synthesis['key_topics']}")
if 'numerical_analysis' in synthesis:
    print(f"Data analysis: {synthesis['numerical_analysis']}")

print("\n✅ Research tools ready for integration!")

## 5. ReAct Agent Implementation

Now let's build our first intelligent agent using the ReAct (Reason + Act) pattern. This agent will use our custom tools to solve complex tasks.

In [ ]:
# Create a comprehensive tool collection
all_tools = [
    simple_calculator,
    weather_simulator, 
    add_numbers,
    multiply_numbers,
    statistical_analysis,
    wikipedia_search,
    knowledge_synthesizer
]

print(f"🔧 Available tools: {len(all_tools)}")
for tool in all_tools:
    print(f"  • {tool.name}: {tool.description.split('.')[0] if tool.description else 'No description'}")

# Mock ReAct Agent Implementation (for demo when no real LLM available)
class MockReActAgent:
    """Mock ReAct agent for demonstration purposes"""
    
    def __init__(self, tools):
        self.tools = {tool.name: tool for tool in tools}
        
    def run(self, query: str) -> Dict[str, Any]:
        """Simulate ReAct reasoning process"""
        
        # Simple heuristic-based tool selection
        query_lower = query.lower()
        
        if "calculate" in query_lower or "add" in query_lower or "multiply" in query_lower:
            if "add" in query_lower:
                # Extract numbers (simplified)
                numbers = [10, 20, 30]  # Mock extraction
                result = self.tools["add_numbers"].invoke({"numbers": numbers})
                return {
                    "thought": "I need to add some numbers together",
                    "action": "add_numbers",
                    "action_input": {"numbers": numbers},
                    "observation": result,
                    "final_answer": f"The sum is {result.get('sum', 'unknown')}"
                }
            
            elif "multiply" in query_lower:
                numbers = [2, 3, 4]  # Mock extraction
                result = self.tools["multiply_numbers"].invoke({"numbers": numbers}) 
                return {
                    "thought": "I need to multiply some numbers",
                    "action": "multiply_numbers", 
                    "action_input": {"numbers": numbers},
                    "observation": result,
                    "final_answer": f"The product is {result.get('product', 'unknown')}"
                }
        
        elif "weather" in query_lower:
            location = "New York"  # Mock extraction
            result = self.tools["weather_simulator"].invoke({"location": location})
            return {
                "thought": "User is asking about weather",
                "action": "weather_simulator",
                "action_input": {"location": location},
                "observation": result,
                "final_answer": f"Weather in {location}: {result['temperature']}°F, {result['condition']}"
            }
        
        elif "search" in query_lower or "research" in query_lower:
            query_term = "artificial intelligence"  # Mock extraction
            result = self.tools["wikipedia_search"].invoke({"query": query_term})
            return {
                "thought": "I need to search for information",
                "action": "wikipedia_search", 
                "action_input": {"query": query_term},
                "observation": result,
                "final_answer": f"Found {result['results_found']} articles about {query_term}"
            }
        
        else:
            return {
                "thought": "This seems like a general question",
                "action": "None",
                "action_input": {},
                "observation": "No specific tool needed",
                "final_answer": f"I understand your question: {query}"
            }

# Create and test the agent
if has_real_llm:
    try:
        # Real ReAct agent with LangChain
        agent = initialize_agent(
            tools=all_tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            max_iterations=3,
            early_stopping_method="generate"
        )
        print("✅ Real ReAct agent created with LangChain")
        agent_type = "real"
    except Exception as e:
        print(f"⚠️  Could not create real agent: {e}")
        agent = MockReActAgent(all_tools)
        agent_type = "mock"
else:
    # Use mock agent for demonstration
    agent = MockReActAgent(all_tools)
    agent_type = "mock"
    print("🎭 Using mock ReAct agent for demonstration")

# Test the agent with various queries
test_queries = [
    "Add the numbers 10, 20, and 30",
    "What's the weather like?", 
    "Search for information about machine learning"
]

print(f"\n🤖 Testing {agent_type.title()} ReAct Agent:")
print("=" * 50)

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 Query {i}: {query}")
    
    try:
        if agent_type == "real":
            response = agent.run(query)
            print(f"💭 Agent Response: {response}")
        else:
            # Mock agent response
            response = agent.run(query)
            print(f"💭 Thought: {response['thought']}")
            print(f"🔧 Action: {response['action']}")
            print(f"👀 Observation: {response['observation']}")
            print(f"✅ Final Answer: {response['final_answer']}")
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n🎉 ReAct agent demonstration complete!")

## 🎯 What You've Accomplished

Congratulations! In this introductory notebook, you've learned how to:

✅ **Set up the development environment** with all required libraries  
✅ **Create custom tools** with proper schemas and type hints  
✅ **Build a math toolkit** with advanced numerical operations  
✅ **Implement research tools** for information gathering  
✅ **Construct a ReAct agent** that can reason and act using tools  

## 📚 Next Steps in the Course

This is just the beginning! The complete course includes:

### 🔧 Advanced Tool Development
- Complex tool architectures
- Error handling and validation
- Tool composition and chaining

### 🏗️ Agent Architectures
- Structured Chat agents for complex I/O
- Plan-and-Execute agents for multi-step tasks
- LangGraph for advanced orchestration

### 🌍 Real-World Applications
- Financial analysis agents
- Customer support automation
- Research and data analysis systems

### 🔍 Best Practices
- Debugging and monitoring techniques
- Performance optimization
- Production deployment strategies

## 🚀 Continue Your Journey

Ready to dive deeper? Check out the other notebooks in this course:
- `01_advanced_tools.ipynb` - Build sophisticated custom tools
- `02_agent_architectures.ipynb` - Explore different agent patterns
- `03_langgraph_orchestration.ipynb` - Master advanced workflows
- `04_real_world_applications.ipynb` - Build production-ready agents

## 💡 Key Takeaways

Remember these fundamental principles:
1. **Tools bridge LLMs and the real world** - They're essential for practical agents
2. **Type hints and error handling matter** - They make tools reliable and debuggable  
3. **Start simple, then scale** - Begin with basic tools before building complex systems
4. **Test everything** - Validate tools independently before agent integration

Happy coding! 🎉